In [3]:
import pyreadr
import polars as pl
import polars.selectors as cs

diffexp_x = pl.read_csv(snakemake.input[0], separator="\t").lazy()
diffexp_y = pl.read_csv(snakemake.input[1], separator="\t").lazy()
label_x = snakemake.params.labels[0]
label_y = snakemake.params.labels[1]

# diffexp_x = pl.read_csv("/projects/koesterlab/moeller-th-liver-diffexp/analysis_2/results/tables/go_terms/etoh_mgl_in_wt_vs_etho_in_wt_liver.go_term_enrichment.gene_fdr_0-05.go_term_sig_study_fdr0-05.tsv", separator="\t").lazy()
# diffexp_y = pl.read_csv("/projects/koesterlab/moeller-th-liver-diffexp/analysis_2/results/tables/go_terms/etoh_t3_in_wt_vs_etoh_in_wt_liver.go_term_enrichment.gene_fdr_0-05.go_term_sig_study_fdr0-05.tsv", separator="\t").lazy()
# label_x = "mgl"
# label_y = "t3"

effect_x_pos = f"positive effect {label_x}"
effect_y_pos = f"positive effect {label_y}"
effect_x_neg = f"negative effect {label_x}"
effect_y_neg = f"negative effect {label_y}"


In [4]:
def extract_study_items(value):
    if value and value.strip() != "":
        gene_values = value.split(', ')
        data = []
        for gene_value in gene_values:
            parts = gene_value.split(':')
            gene = parts[0]
            val = float(parts[1])
            data.append({"gene": gene, "value": val})
        return data
    else:
        return []

def calculate_sums(parsed_terms):
    positive_sum = sum(item['value'] for item in parsed_terms if item['value'] > 0)
    negative_sum = sum(item['value'] for item in parsed_terms if item['value'] < 0)
    return positive_sum, negative_sum

In [5]:
def prepare(df):
    # Select necessary columns
    df = df.select([
        cs.by_name("GO", "term", "p_uncorrected", "p_fdr_bh", "study_items_sig_terms")
    ])
    
    # map_elements functions to columns using with_columns
    df = df.with_columns([
        pl.col("study_items_sig_terms").map_elements(extract_study_items).alias("parsed_terms")])
    df = df.with_columns([
        pl.col("parsed_terms").map_elements(lambda x: calculate_sums(x)[0]).alias("cumulative_b_scores_positive"),
        pl.col("parsed_terms").map_elements(lambda x: calculate_sums(x)[1]).alias("cumulative_b_scores_negative")
    ])
    
    return df

In [6]:
prepared_diffexp_x = prepare(diffexp_x)
prepared_diffexp_y = prepare(diffexp_y)
combined = prepared_diffexp_x.join(
    prepared_diffexp_y, on=["GO", "term"], suffix="_y"
).with_columns(
    pl.min_horizontal("p_fdr_bh", "p_fdr_bh_y").alias("p_fdr_bh_min")
).filter(
    pl.col("p_fdr_bh_min") <= 0.05
).rename(
    {
        "cumulative_b_scores_positive": effect_x_pos,
        "cumulative_b_scores_positive_y": effect_y_pos,
        "cumulative_b_scores_negative": effect_x_neg,
        "cumulative_b_scores_negative_y": effect_y_neg,
        "p_fdr_bh_min": "min p_fdr_bh",
    }
).collect()

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredict

In [7]:
combined

GO,term,p_uncorrected,p_fdr_bh,study_items_sig_terms,parsed_terms,positive effect mgl,negative effect mgl,p_uncorrected_y,p_fdr_bh_y,study_items_sig_terms_y,parsed_terms_y,positive effect t3,negative effect t3,min p_fdr_bh
str,str,f64,f64,str,list[struct[2]],f64,f64,f64,f64,str,list[struct[2]],f64,f64,f64
"""GO:0006629""","""lipid metabolic process""",2.7545e-61,3.4109e-57,"""Fah:0.508, Dhcr24:1.27, Lypla1…","[{""Fah"",0.508}, {""Dhcr24"",1.27}, … {""Cyp2c38"",2.2}]",184.10933,-135.15124,5.5125e-38,3.4131e-34,"""Scp2:-0.752, Ces1g:-0.8, Plbd1…","[{""Scp2"",-0.752}, {""Ces1g"",-0.8}, … {""Neu1"",0.628}]",58.609,-66.752,3.4109e-57
"""GO:0007608""","""sensory perception of smell""",6.1322e-42,1.8984e-38,"""Or2ag20:1.69, Or5d18:1.82, Or4…","[{""Or2ag20"",1.69}, {""Or5d18"",1.82}, … {""Or1j15"",0.91}]",24.873,-15.23,3.8951e-11,2.8372e-8,"""Or1e32:2.9, Or4f14b:1.1, B2m:0…","[{""Or1e32"",2.9}, {""Or4f14b"",1.1}, … {""Or10ag59"",2.27}]",6.888,0.0,1.8984e-38
"""GO:0006631""","""fatty acid metabolic process""",7.0720e-37,1.7514e-33,"""Lypla1:-0.484, Acads:1.13, C3:…","[{""Lypla1"",-0.484}, {""Acads"",1.13}, … {""Pecr"",0.701}]",60.493,-50.0166,6.0084e-17,9.3002e-14,"""Acsl1:-0.815, Fads1:0.544, Lpi…","[{""Acsl1"",-0.815}, {""Fads1"",0.544}, … {""Fads2"",1.26}]",17.256,-32.78,1.7514e-33
"""GO:0007186""","""G protein-coupled receptor sig…",2.8894e-32,3.9755e-29,"""Or2ag20:1.69, Apoe:0.842000000…","[{""Or2ag20"",1.69}, {""Apoe"",0.842}, … {""Apoa1"",1.44}]",70.384,-40.467,0.000194,0.013482,"""Ccl5:3.04, Adgra3:-1.11, Ccl9:…","[{""Ccl5"",3.04}, {""Adgra3"",-1.11}, … {""Cxcl10"",4.18}]",50.685,-11.693,3.9755e-29
"""GO:0002376""","""immune system process""",1.4476e-30,1.7925e-27,"""Hmgb2:-1.74, Cadm1:1.45, Serpi…","[{""Hmgb2"",-1.74}, {""Cadm1"",1.45}, … {""Rsad2"",3.72}]",182.7968,-54.5817,8.6964e-46,1.0769e-41,"""Iigp1:1.23, Themis2:3.46, C1ra…","[{""Iigp1"",1.23}, {""Themis2"",3.46}, … {""Slc15a3"",1.79}]",162.701,-15.033,1.0769e-41
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""GO:0004029""","""aldehyde dehydrogenase (NAD+) …",0.002638,0.034555,"""Aldh1a1:-0.447, Aldh1a7:0.728,…","[{""Aldh1a1"",-0.447}, {""Aldh1a7"",0.728}, … {""Aldh7a1"",-0.936}]",1.8012,-1.383,0.000093,0.005015,"""Aldh7a1:-0.81, Aldh2:-0.586, A…","[{""Aldh7a1"",-0.81}, {""Aldh2"",-0.586}, … {""Aldh1a1"",-0.588}]",0.0,-3.715,0.005015
"""GO:0008391""","""arachidonic acid monooxygenase…",0.002638,0.034555,"""Cyp4a10:0.716, Cyp2d22:-0.0859…","[{""Cyp4a10"",0.716}, {""Cyp2d22"",-0.0859}, … {""Cyp4a31"",2.49}]",3.206,-12.5359,2.4159e-7,0.000027,"""Cyp2d22:-0.741, Cyp4a14:1.88, …","[{""Cyp2d22"",-0.741}, {""Cyp4a14"",1.88}, … {""Cyp4a10"",0.793}]",5.633,-14.391,0.000027
"""GO:0008047""","""enzyme activator activity""",0.002749,0.035898,"""Krtcap2:0.353, Timp2:0.6559999…","[{""Krtcap2"",0.353}, {""Timp2"",0.656}, … {""Dtx3l"",0.897}]",5.568,-3.548,0.000328,0.013481,"""Apobec1:2.0, Arl1:0.721, Cd44:…","[{""Apobec1"",2.0}, {""Arl1"",0.721}, … {""Alox5ap"",1.56}]",7.785,0.0,0.013481


In [10]:
import altair as alt
import sys
from IPython.display import display
# we cannot use vegafusion here because it makes the point selection impossible since
# it prunes the required ext_gene column
#alt.data_transformers.enable("vegafusion")
xlabel = f"effect {label_x}"
ylabel = f"effect {label_y}"
df_filtered = combined.filter((pl.col(effect_x_pos) != 0) & (pl.col(effect_y_pos) != 0) & (pl.col(effect_x_neg) != 0) & (pl.col(effect_y_neg) != 0))
df_filtered = df_filtered.with_columns(
    pl.max_horizontal(abs(pl.col(effect_x_pos) - pl.col(effect_y_pos)), abs(pl.col(effect_x_neg) - pl.col(effect_y_neg))).alias("difference")
)
combined_sorted = df_filtered.sort("difference", descending=True)
combined_pd = combined_sorted.select(
    pl.col("GO", "term", "min p_fdr_bh", effect_x_pos, effect_y_pos, effect_x_neg, effect_y_neg)
).to_pandas()
# combined_pd = combined_pd["GO"].astype(str)

combined_pd.to_csv(snakemake.output[0], sep="\t", index=False)
# combined_pd.to_csv("/projects/koesterlab/moeller-th-liver-diffexp/analysis_2/results/tables/go_terms/meta_compare_etoh_wt_mgl_vs_t3.tsv", sep="\t", index=False)

point_selector = alt.selection_point(fields=["term"], empty=False)


def plot(df, effect_x, effect_y, title, xlabel, ylabel):
    # Filter out rows where either effect_x or effect_y is zero because of logarithmic scale

    df = df_filtered.select(
        pl.col("GO", "term", "min p_fdr_bh", effect_x, effect_y)
    ).to_pandas()


    effects = df_filtered.select(pl.col(effect_x), pl.col(effect_y))
    min_value = effects.min().min_horizontal()[0]
    max_value = effects.max().max_horizontal()[0]

    alt.data_transformers.disable_max_rows()


    points = alt.Chart(df).mark_circle(size=15, tooltip={"content": "data"}).encode(
        alt.X(effect_x, title=xlabel, scale=alt.Scale(type='log', nice=False), axis=alt.Axis(grid=False)),
        alt.Y(effect_y, title=ylabel, scale=alt.Scale(type='log', nice=False), axis=alt.Axis(grid=False)),
        alt.Color("min p_fdr_bh", scale=alt.Scale(scheme="viridis")),
        opacity=alt.value(0.5),
    )

    line = alt.Chart(
        pl.DataFrame({effect_x: [min_value, max_value], effect_y: [min_value, max_value]})
    ).mark_line(color="lightgrey").encode(
        x=effect_x,
        y=effect_y,
        strokeDash=alt.value([5, 5]),
    )
    
    text_background = alt.Chart(df).mark_text(
        align="left",
        baseline="middle",
        dx=5,
        dy=-5,
        fill='white',
        stroke='white',
        strokeWidth=5,
    ).encode(
        x=effect_x,
        y=effect_y,
        text=alt.condition(point_selector, "term", alt.value("")),
    )

    text = alt.Chart(df).mark_text(
        align="left",
        baseline="middle",
        dx=5,
        dy=-5,  
           
    ).encode(
        x=effect_x,
        y=effect_y,
        text=alt.condition(point_selector, "term", alt.value("")),
    )
    

    chart = alt.layer(line, points, text_background, text).add_params(
        point_selector
    ).properties(
        title=title
    ).interactive()
    return chart

# Update the plot function calls to include the logarithmic scale and filter out zero values
positive_chart = plot(combined_pd, effect_x_pos, effect_y_pos, "Positive effect", xlabel, ylabel)
negative_chart = plot(combined_pd, effect_x_neg, effect_y_neg, "Negative effect", xlabel, ylabel)

final_chart = alt.hconcat(positive_chart, negative_chart).resolve_scale(
    color='shared'
)

display(final_chart)

# final_chart.save("/projects/koesterlab/moeller-th-liver-diffexp/analysis_2/test.html", inline=True)
final_chart.save(snakemake.output[1], inline=True)

alt.HConcatChart(...)